In [1]:
import keras

Using TensorFlow backend.
/Users/hetty/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout, Activation
from datetime import date, timedelta
import pandas as pd
import numpy as np

In [5]:
# construct model
model = Sequential()
model.add(LSTM(30, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=False))
model.add(Dense(30))
model.add(Dropout(.2))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [4]:
print('Loading Data')
df_train = pd.read_csv('train.csv',dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
    parse_dates=["date"], skiprows=range(1, 101688780) )
# read data starting from 2017-1-1

Loading Data


In [5]:
df_2017 = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack().fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [6]:
items = pd.read_csv("items.csv")
stores = pd.read_csv('stores.csv')

In [29]:
df_2017_reset = df_2017.reset_index()
temp_a = pd.merge(df_2017_reset, items, on='item_nbr')
temp_b = pd.merge(temp_a, stores, on='store_nbr')
train_2017 = temp_b.set_index(["store_nbr", "item_nbr"])

In [59]:
promo_2017_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack().fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

In [60]:
promo_2017_train = promo_2017_train.reindex(train_2017.index)

In [61]:
df_test = pd.read_csv("test.csv", dtype={'onpromotion': bool}, parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])

In [62]:
promo_2017_test = df_test[["onpromotion"]].unstack().fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

In [63]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [64]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [66]:
promo_2017 = promo_2017.iloc[:,151:]

In [68]:
items = pd.read_csv("items.csv").set_index("item_nbr")
items = items.reindex(train_2017.index.get_level_values(1))

In [78]:
train_start_date = date(2017,7,5)
last_day = train_2017[pd.date_range(train_start_date - timedelta(days=1), periods=22, freq='D')]

In [86]:
last_day = np.reshape(np.array(last_day), (last_day.shape[0],last_day.shape[1],1))

In [131]:
last_3_day = train_2017[pd.date_range(train_start_date - timedelta(days=3), periods=24, freq='D')]
last_3_day = last_3_day.rolling(3,axis=1).mean()
last_3_day = np.reshape(np.array(last_3_day.iloc[:,2:]), 
                        (last_3_day.shape[0],last_3_day.shape[1]-2,1))

In [133]:
last_7_day = train_2017[pd.date_range(train_start_date - timedelta(days=7), periods=28, freq='D')]
last_7_day = last_7_day.rolling(7,axis=1).mean()
last_7_day = np.reshape(np.array(last_7_day.iloc[:,6:]),
                        (last_7_day.shape[0], last_7_day.shape[1]-6, 1))
last_7_day.shape

(167515, 22, 1)

In [135]:
last_14_day = train_2017[pd.date_range(train_start_date - timedelta(days=14), periods=35, freq='D')]
last_14_day = last_14_day.rolling(14,axis=1).mean()
last_14_day = np.reshape(np.array(last_14_day.iloc[:,13:]),
                         (last_14_day.shape[0],last_14_day.shape[1]-13,1))
last_14_day.shape

(167515, 22, 1)

In [138]:
last_30_day = train_2017[pd.date_range(train_start_date - timedelta(days=30), periods=51, freq='D')]
last_30_day = last_30_day.rolling(30,axis=1).mean()
last_30_day = np.reshape(np.array(last_30_day.iloc[:,29:]),
                         (last_30_day.shape[0],last_30_day.shape[1]-29,1))
last_30_day.shape

(167515, 22, 1)

In [139]:
dtrain = np.concatenate((last_day,last_3_day,last_7_day,last_14_day,last_30_day),axis=2)

In [141]:
promo_2017 = promo_2017.astype(np.uint8)

In [144]:
promo_today = promo_2017[pd.date_range(train_start_date, periods=22, freq='D')]
promo_today = np.reshape(np.array(promo_today), (promo_today.shape[0],promo_today.shape[1],1))

In [146]:
promo_today.shape

(167515, 22, 1)

In [153]:
num = [3,7,14,30]

for i in num:
    promo_tem = promo_2017[pd.date_range(train_start_date - timedelta(days=i),periods=21+i, freq='D')]
    promo_tem = promo_tem.rolling(i,axis=1).sum()
    promo_tem = np.reshape(np.array(promo_tem.iloc[:,i-1:]),
                           (promo_tem.shape[0],promo_tem.shape[1]-i+1,1))
    promo_today = np.concatenate((promo_today,promo_tem),axis=2)

In [159]:
X_train = np.concatenate((dtrain,promo_today),axis=2)

In [161]:
Y_train = train_2017[pd.date_range(date(2017,7,26), periods=16)].values
Y_train.shape

(167515, 16)

In [162]:
# construct model
model = Sequential()
model.add(LSTM(22, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=False))
model.add(Dense(22))
model.add(Dropout(.2))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [163]:
import random

In [168]:
random.seed(100)
random.shuffle(X_train)

In [169]:
random.seed(100)
random.shuffle(Y_train)

In [214]:
x_train = X_train[:140000,:,:]
y_train = Y_train[:140000,:]

In [215]:
x_val = X_train[140000:,:,:]
y_val = Y_train[140000:,:]

In [179]:
sample_weights = np.array(items['perishable']*0.25+1)

In [189]:
random.seed(100)
random.shuffle(sample_weights)
sample_weights_train = sample_weights[:140000]

In [192]:
test_date = date(2017,7,26)
last_day = train_2017[pd.date_range(test_date - timedelta(days=1), periods=22, freq='D')]
last_day = np.reshape(np.array(last_day), (last_day.shape[0],last_day.shape[1],1))

num = [3,7,14,30]

for i in num:
    sales_tem = train_2017[pd.date_range(test_date - timedelta(days=i),periods=21+i, freq='D')]
    sales_tem = sales_tem.rolling(i,axis=1).mean()
    sales_tem = np.reshape(np.array(sales_tem.iloc[:,i-1:]),
                             (sales_tem.shape[0],sales_tem.shape[1]-i+1,1))
    last_day = np.concatenate((last_day,sales_tem),axis=2)

In [194]:
promo_today = promo_2017[pd.date_range(test_date, periods=22, freq='D')]
promo_today = np.reshape(np.array(promo_today), (promo_today.shape[0],promo_today.shape[1],1))

for i in num:
    promo_tem = promo_2017[pd.date_range(test_date - timedelta(days=i),periods=21+i, freq='D')]
    promo_tem = promo_tem.rolling(i,axis=1).sum()
    promo_tem = np.reshape(np.array(promo_tem.iloc[:,i-1:]),
                           (promo_tem.shape[0],promo_tem.shape[1]-i+1,1))
    promo_today = np.concatenate((promo_today,promo_tem),axis=2)

In [196]:
X_test = np.concatenate((last_day,promo_today),axis=2)

In [246]:
val_pred = []
test_pred = []

In [247]:
for i in range(16):
    
    promo_today = promo_2017[date(2017,7,26) + timedelta(days=i)].values
    random.seed(100)
    random.shuffle(promo_today)
    
    x_train[:,21,5] = promo_today[:140000]
    x_val[:,21,5] = promo_today[140000:]
    
    model.fit(x_train, y_train[:,i], sample_weight=sample_weights_train,batch_size=54,
              epochs=8, verbose=1, shuffle=False, 
              validation_data=(x_val,y_val[:,i],sample_weights[140000:]))
    
    val_pred.append(model.predict(x_val))
    
    promo_today = promo_2017[date(2017,8,16) + timedelta(days=i)].values
    X_test[:,21,5] = promo_today
    
    test_pred.append(model.predict(X_test))   

Train on 140000 samples, validate on 27515 samples
Epoch 1/8
140000/140000 [==============================] - 72s 516us/step - loss: 0.3557 - mean_squared_error: 0.3367 - val_loss: 0.3464 - val_mean_squared_error: 0.3285
Epoch 2/8
140000/140000 [==============================] - 70s 500us/step - loss: 0.3469 - mean_squared_error: 0.3284 - val_loss: 0.3444 - val_mean_squared_error: 0.3266
Epoch 3/8
140000/140000 [==============================] - 63s 453us/step - loss: 0.3440 - mean_squared_error: 0.3257 - val_loss: 0.3414 - val_mean_squared_error: 0.3237
Epoch 4/8
140000/140000 [==============================] - 59s 419us/step - loss: 0.3417 - mean_squared_error: 0.3236 - val_loss: 0.3402 - val_mean_squared_error: 0.3226
Epoch 5/8
140000/140000 [==============================] - 58s 413us/step - loss: 0.3405 - mean_squared_error: 0.3225 - val_loss: 0.3400 - val_mean_squared_error: 0.3224
Epoch 6/8
140000/140000 [==============================] - 58s 415us/step - loss: 0.3393 - mean_squ

140000/140000 [==============================] - 55s 393us/step - loss: 0.4230 - mean_squared_error: 0.4008 - val_loss: 0.4500 - val_mean_squared_error: 0.4263
Epoch 6/8
140000/140000 [==============================] - 55s 394us/step - loss: 0.4219 - mean_squared_error: 0.3998 - val_loss: 0.4497 - val_mean_squared_error: 0.4260
Epoch 7/8
140000/140000 [==============================] - 54s 388us/step - loss: 0.4204 - mean_squared_error: 0.3983 - val_loss: 0.4479 - val_mean_squared_error: 0.4243
Epoch 8/8
140000/140000 [==============================] - 55s 396us/step - loss: 0.4197 - mean_squared_error: 0.3976 - val_loss: 0.4485 - val_mean_squared_error: 0.4249
Train on 140000 samples, validate on 27515 samples
Epoch 1/8
140000/140000 [==============================] - 56s 400us/step - loss: 0.4149 - mean_squared_error: 0.3929 - val_loss: 0.4282 - val_mean_squared_error: 0.4052
Epoch 2/8
140000/140000 [==============================] - 56s 398us/step - loss: 0.4110 - mean_squared_error

In [248]:
from sklearn.metrics import mean_squared_error

In [249]:
val_pred_new = np.array(val_pred).transpose()[0]

In [250]:
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred_new))**0.5)

Validation mse: 0.622845742629


In [251]:
pred_result = np.array(test_pred).transpose()[0]

In [252]:
pred_result.shape

(167515, 16)

In [253]:
df_preds = pd.DataFrame(pred_result, index=train_2017.index, columns=pd.date_range("2017-08-16", periods=16)
                       ).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [254]:
submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 10000)
submission.shape

(3370464, 2)

In [255]:
submission.to_csv('try_final_2.csv', float_format='%.4f', index=None)